In [34]:
import os
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
pwd

'/home/ubuntu/Github/Build-Week-4'

In [3]:
ls

 LAlistings.csv                              berlin/
 LAsummary.csv                               berlinlistings.csv
'Pandas Profiling Report — AirBNB_LA.html'   listings.csv.gz
 Project4-engineering.ipynb


In [4]:
listings = pd.read_csv("../Build-Week-4/LAlistings.csv")

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
listings.shape

(40125, 106)

In [6]:
LAsummary = pd.read_csv("../Build-Week-4/LAsummary.csv")
LAsummary.sample(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
11358,16108348,Los Angeles Palace -10 Beds - Sleep 20 -6 Bedr...,88810222,Patrick,Other Cities,Lancaster,34.68854,-118.07529,Entire home/apt,199,2,85,2019-11-10,2.31,3,149
24506,29127929,The Surfrider Villa - Malibu Road - Amazing Views,24303784,Stuart,Other Cities,Malibu,34.03243,-118.69913,Entire home/apt,1200,5,19,2019-12-01,2.71,17,63
18148,22636639,Fully equipped ocean view apartment with parking!,69131370,Inge,City of Los Angeles,Venice,33.99112,-118.47663,Entire home/apt,185,1,131,2019-10-29,5.79,1,0
23162,27833696,Great 1 bed unit Los Angeles /San Fernando Valley,16672866,Carlos,City of Los Angeles,Panorama City,34.24149,-118.45146,Entire home/apt,79,2,4,2019-10-12,0.27,17,309
12547,17585662,Stanley House Room #2 with Private bathroom,118689294,Stanley,City of Los Angeles,Koreatown,34.05571,-118.31893,Private room,45,30,23,2019-10-04,0.76,4,179


In [7]:
LAsummary.shape

(40125, 16)

In [8]:
features = listings.columns.values
features

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access',
       'interaction', 'house_rules', 'thumbnail_url', 'medium_url',
       'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bath

In [9]:
listings.describe()

,id,scrape_id,thumbnail_url,medium_url,xl_picture_url,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,4.012500e+04,4.012500e+04,0.0,0.0,0.0,4.012500e+04,0.0,39690.000000,39690.000000,40125.000000,...,31450.000000,31431.000000,31450.000000,31426.000000,31419.000000,40125.000000,40125.000000,40125.000000,40125.000000,31980.000000
mean,2.357154e+07,2.019121e+13,NaN,NaN,NaN,8.692751e+07,NaN,22.942882,22.942882,34.050199,...,9.416502,9.751583,9.730397,9.699103,9.439352,7.659539,5.719576,1.261707,0.477782,1.870650
std,1.181384e+07,1.216421e+01,NaN,NaN,NaN,8.861631e+07,NaN,146.223989,146.223989,0.122755,...,0.990258,0.761890,0.789833,0.721369,0.921400,19.448746,18.946280,3.141152,3.050245,1.971808
min,1.090000e+02,2.019121e+13,NaN,NaN,NaN,5.900000e+01,NaN,0.000000,0.000000,33.338480,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,1.450446e+07,2.019121e+13,NaN,NaN,NaN,1.353996e+07,NaN,1.000000,1.000000,34.002190,...,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.360000
50%,2.445198e+07,2.019121e+13,NaN,NaN,NaN,4.929784e+07,NaN,2.000000,2.000000,34.062170,...,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,1.000000,0.000000,0.000000,1.150000
75%,3.405760e+07,2.019121e+13,NaN,NaN,NaN,1.462027e+08,NaN,6.000000,6.000000,34.104100,...,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000,3.000000,1.000000,0.000000,2.820000
max,4.060385e+07,2.019121e+13,NaN,NaN,NaN,3.145798e+08,NaN,1759.000000,1759.000000,34.811190,...,10.000000,10.000000,10.000000,10.000000,10.000000,185.000000,185.000000,36.000000,44.000000,44.380000


In [10]:
listings.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
name                                             object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 106, dtype: object

In [11]:
# !pip install pandas-profiling

In [30]:
# import pandas_profiling

/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [35]:
# profile = listings.profile_report(title='Pandas_Profiling_Report')
# profile.to_file(output_file="Pandas Profiling Report — AirBNB_LA.html")

In [12]:
pd.set_option('display.max_columns', None)

In [13]:
listings.describe()

,id,scrape_id,thumbnail_url,medium_url,xl_picture_url,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,4.012500e+04,4.012500e+04,0.0,0.0,0.0,4.012500e+04,0.0,39690.000000,39690.000000,40125.000000,40125.000000,40125.000000,40100.000000,40055.000000,40033.000000,292.000000,40125.000000,40125.000000,40125.000000,40125.000000,40125.000000,4.012500e+04,4.012500e+04,40125.000000,4.012500e+04,40125.000000,40125.000000,40125.000000,40125.000000,40125.000000,40125.000000,31469.000000,31451.000000,31450.000000,31431.000000,31450.000000,31426.000000,31419.000000,40125.000000,40125.000000,40125.000000,40125.000000,31980.000000
mean,2.357154e+07,2.019121e+13,NaN,NaN,NaN,8.692751e+07,NaN,22.942882,22.942882,34.050199,-118.319154,3.690966,1.481471,1.450006,1.994330,1021.520548,1.962168,10.867688,677.527751,10.268735,11.265794,5.894282e+05,8.035098e+05,10.874378,7.925498e+05,11.508486,27.764187,45.447751,158.237458,34.082069,13.452411,94.325590,9.616324,9.416502,9.751583,9.730397,9.699103,9.439352,7.659539,5.719576,1.261707,0.477782,1.870650
std,1.181384e+07,1.216421e+01,NaN,NaN,NaN,8.861631e+07,NaN,146.223989,146.223989,0.122755,0.163146,2.696065,1.007890,1.133405,1.651987,1021.075174,1.820588,20.848665,5046.828589,20.287699,21.456872,3.555203e+07,4.151376e+07,20.869264,4.096112e+07,11.477449,23.213899,35.244582,139.786661,61.568212,20.859674,8.777228,0.886665,0.990258,0.761890,0.789833,0.721369,0.921400,19.448746,18.946280,3.141152,3.050245,1.971808
min,1.090000e+02,2.019121e+13,NaN,NaN,NaN,5.900000e+01,NaN,0.000000,0.000000,33.338480,-118.934220,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,1.450446e+07,2.019121e+13,NaN,NaN,NaN,1.353996e+07,NaN,1.000000,1.000000,34.002190,-118.422220,2.000000,1.000000,1.000000,1.000000,400.000000,1.000000,1.000000,31.000000,1.000000,1.000000,3.500000e+01,4.000000e+01,1.000000,4.000000e+01,0.000000,0.000000,1.000000,7.000000,1.000000,0.000000,93.000000,9.000000,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.360000
50%,2.445198e+07,2.019121e+13,NaN,NaN,NaN,4.929784e+07,NaN,2.000000,2.000000,34.062170,-118.342290,3.000000,1.000000,1.000000,1.000000,750.000000,1.000000,2.000000,1125.000000,2.000000,3.000000,1.125000e+03,1.125000e+03,2.200000,1.125000e+03,8.000000,30.000000,54.000000,133.000000,8.000000,3.000000,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,1.000000,0.000000,0.000000,1.150000
75%,3.405760e+07,2.019121e+13,NaN,NaN,NaN,1.462027e+08,NaN,6.000000,6.000000,34.104100,-118.252710,5.000000,2.000000,2.000000,2.000000,1200.000000,2.000000,30.000000,1125.000000,20.000000,30.000000,1.125000e+03,1.125000e+03,28.800000,1.125000e+03,23.000000,51.000000,80.000000,317.000000,39.000000,19.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000,3.000000,1.000000,0.000000,2.820000
max,4.060385e+07,2.019121e+13,NaN,NaN,NaN,3.145798e+08,NaN,1759.000000,1759.000000,34.811190,-117.647740,24.000000,16.000000,21.000000,50.000000,7000.000000,24.

In [14]:
listings.sample(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
11822,16602846,https://www.airbnb.com/rooms/16602846,20191205175409,2019-12-06,"APT On Colorado BLVD, Pasadena",- In quiet Neighborhood on Colorado Blvd. - 10...,NaN,- In quiet Neighborhood on Colorado Blvd. - 10...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/f2a08a49-9...,NaN,13689840,https://www.airbnb.com/users/show/13689840,Analisse,2014-03-30,US,NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/13689840/prof...,https://a0.muscache.com/im/users/13689840/prof...,Pasadena,1.0,1.0,"['email', 'phone', 'google', 'jumio', 'governm...",t,t,"Pasadena, CA, United States",Pasadena,Pasadena,Other Cities,Pasadena,CA,91107,Los Angeles,"Pasadena, CA",US,United States,34.14586,-118.10713,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,Breakfast,""Family/kid friendly"",""Smoke d...",NaN,$150.00,NaN,NaN,NaN,NaN,1,$0.00,1,2,1,1,2,2,1.0,2.0,36 months ago,t,28,58,88,363,2019-12-06,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,"{Pasadena,"" CA""}",f,f,flexible,f,f,1,0,1,0,NaN
22627,27316115,https://www.airbnb.com/rooms/27316115,20191205175409,2019-12-07,OceanMalibu Desired 3 tier Estate LA Luxuries,"We operate a Luxury Estates, Services, and Rec...","This 2,600 square foot home has a state of the...","We operate a Luxury Estates, Services, and Rec...",none,NaN,NaN,NaN,NaN,NaN,No loud noise past 10pm. City law ordinance.,NaN,NaN,https://a0.muscache.com/im/pictures/83b2c710-0...,NaN,94261069,https://www.airbnb.com/users/show/94261069,LA Luxuries,2016-09-08,"Beverly Hills, California, United States","We are a Luxury Estates, Events and recreation...",within an hour,90%,NaN,f,https://a0.muscache.com/im/pictures/user/7b5e1...,https://a0.muscache.com/im/pictures/user/7b5e1...,Beverly Hills,10.0,10.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Malibu, CA, United States",Malibu,Malibu,Other Cities,Malibu,CA,90265,Malibu,"Malibu, CA",US,United States,34.02614,-118.83697,f,House,Entire home/apt,6,4.0,4.0,3.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Free park...",NaN,"$3,500.00",NaN,NaN,"$5,000.00",$600.00,5,$100.00,31,1125,31,31,1125,1125,31.0,1125.0,12 months ago,t,30,60,89,89,2019-12-07,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,10,10,0,0,NaN
30195

In [16]:
neededcols=[
'id', 'summary', 'space', 'description', 'host_since', 'host_is_superhost', 'host_listings_count', 'host_verifications',
'host_has_profile_pic', 'host_identity_verified','neighbourhood_cleansed','city', 'zipcode', 'latitude',
'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
'bed_type', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 
'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification'
]

grpcols = [
'price', 'beds', 'bedrooms', 'bathrooms', 'zipcode', 'neighbourhood',
'property_type', 'room_type', 'accommodates', 'guests_included',
'minimum_nights', 'instant_bookable']

goodcols = ['']

len(neededcols)

41

In [17]:
list1 = listings[neededcols]

list1.shape

(40125, 41)

In [18]:
list1.sample(5)

,id,summary,space,description,host_since,host_is_superhost,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,city,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
30824,34625831,Comfortable bed in a beautiful room in a clean...,NaN,Comfortable bed in a beautiful room in a clean...,2014-09-15,t,2.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Reseda,Los Angeles,91335,34.19726,-118.55169,t,House,Private room,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",$40.00,$100.00,$27.00,1,$0.00,5,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f
29629,33690300,Stay in a multi-million dollar neighborhood an...,NaN,Stay in a multi-million dollar neighborhood an...,2012-10-12,f,63.0,"['email', 'phone', 'manual_online', 'reviews',...",t,t,Downtown,Los Angeles,90015,34.03938,-118.26829,f,Loft,Entire home/apt,4,1.0,0.0,0.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,""Pets...",$200.00,$100.00,$150.00,1,$0.00,3,100.0,10.0,10.0,10.0,10.0,10.0,9.0,t,strict_14_with_grace_period,f,f
6304,9080284,Beautiful House in one of the most desired Hol...,Very cool Hollywood flat. Perfect for 2 people...,Beautiful House in one of the most desired Hol...,2014-08-06,t,4.0,"['email', 'phone', 'reviews', 'offline_governm...",t,f,Hollywood,Los Angeles,90038,34.09091,-118.33276,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Free street parking...",$109.00,$200.00,$95.00,1,$0.00,195,97.0,10.0,10.0,10.0,10.0,10.0,10.0,t,strict_14_with_grace_period,f,f
6629,9547801,"Spacious 3 BR hm w/lg. kitchen, dining area & ...",This house is considered a duplex consisting o...,"Spacious 3 BR hm w/lg. kitchen, dining area & ...",2015-11-19,f,1.0,"['email', 'phone', 'reviews']",t,f,Long Beach,Long Beach,90803,33.74769,-118.12128,t,House,Entire home/apt,8,3.0,3.0,4.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",$500.00,"$2,500.00",$150.00,6,$50.00,18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,f,f
9449,13788737,Ideal for Traveling Nurses and professionals o...,Ideal for Traveling nurses or traveling profes...,Ideal for Traveling Nurses and professionals o...,2013-05-07,f,2.0,"['email', 'phone', 'reviews', 'kba']",t,t,Long Beach,Long Beach,90802,33.77125,-118.19525,t,Condominium,Entire home/apt,2,1.0,0.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",$97.00,$500.00,$93.00,2,$20.00,183,93.0,10.0,10.0,10.0,10.0,9.0,10.0,t,moderate,f,f


In [19]:
list1.isnull().sum()

id                                      0
summary                              2104
space                               10436
description                          1406
host_since                            435
host_is_superhost                     435
host_listings_count                   435
host_verifications                      0
host_has_profile_pic                  435
host_identity_verified                435
neighbourhood_cleansed                  0
city                                   23
zipcode                               464
latitude                                0
longitude                               0
is_location_exact                       0
property_type                           0
room_type                               0
accommodates                            0
bathrooms                              25
bedrooms                               70
beds                                   92
bed_type                                0
amenities                         

In [20]:
list1.dtypes

id                                    int64
summary                              object
space                                object
description                          object
host_since                           object
host_is_superhost                    object
host_listings_count                 float64
host_verifications                   object
host_has_profile_pic                 object
host_identity_verified               object
neighbourhood_cleansed               object
city                                 object
zipcode                              object
latitude                            float64
longitude                           float64
is_location_exact                    object
property_type                        object
room_type                            object
accommodates                          int64
bathrooms                           float64
bedrooms                            float64
beds                                float64
bed_type                        

In [21]:
descript_feats = ['summary', 'space', 'description']

bool_feats = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'is_location_exact',
              'require_guest_profile_picture', 'require_guest_phone_verification']

date_feats = ['host_since']


In [22]:
for feat in descript_feats:
    list1[feat].replace(np.nan, 'No information given', inplace=True)

for feat in bool_feats:
    list1[feat].replace(np.nan, list1.mode()[feat][0], inplace=True)
    

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [42]:
list1.isnull().sum()

id                                     0
summary                                0
space                                  0
description                            0
host_since                           435
host_is_superhost                      0
host_listings_count                  435
host_verifications                     0
host_has_profile_pic                   0
host_identity_verified                 0
neighbourhood_cleansed                 0
city                                  23
zipcode                              464
latitude                               0
longitude                              0
is_location_exact                      0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                             25
bedrooms                              70
beds                                  92
bed_type                               0
amenities                              0
price           

In [31]:
list1['wifi'] = list1['amenities'].str.contains('Wifi')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
list1['wifi'].value_counts()

True     39144
False      981
Name: wifi, dtype: int64

In [144]:
list2 = list1.copy()

In [145]:
list2['amenities'] =list2['amenities'].str.split(",")

In [146]:
list2['amenities'].head()

0    [{TV, "Cable TV", Internet, Wifi, "Air conditi...
1    [{TV, "Cable TV", Internet, Wifi, "Air conditi...
2    [{Internet, Wifi, "Air conditioning", "Wheelch...
3    [{Internet, Wifi, Kitchen, Heating, Washer, Dr...
4    [{TV, "Cable TV", Internet, Wifi, "Air conditi...
Name: amenities, dtype: object

In [147]:
emptylist = []

for i in list2['amenities']:
    emptylist.append(len(i))

list2['amenities_count'] = emptylist

list2['amenities_count']

0        32
1        41
2        44
3        12
4        20
         ..
40120    21
40121    16
40122    15
40123    23
40124    21
Name: amenities_count, Length: 40125, dtype: int64

In [148]:
list2['amenities_count'].max()

131

In [149]:
list2[list2['amenities_count'] == 131]['amenities']

30400    [{TV, "Cable TV", Internet, Wifi, "Air conditi...
Name: amenities, dtype: object

In [150]:
list2[list2['amenities_count'] == 131]['amenities'].values

array([list(['{TV', '"Cable TV"', 'Internet', 'Wifi', '"Air conditioning"', 'Pool', 'Kitchen', '"Free parking on premises"', 'Gym', 'Elevator', '"Garage parking"', '"Hot tub"', '"Indoor fireplace"', '"Suitable for events"', 'Washer', 'Dryer', '"Smoke detector"', '"Carbon monoxide detector"', 'Iron', 'Bathtub', '"Bath towel"', '"Hand or paper towel"', '"Toilet paper"', 'Microwave', '"Coffee maker"', 'Refrigerator', 'Dishwasher', '"Dishes and silverware"', '"Cooking basics"', 'Oven', 'Stove', '"BBQ grill"', '"Patio or balcony"', '"Garden or backyard"', '"Handheld shower head"', '"Ceiling fan"', '"Central air conditioning"', '"Walk in closet"', 'Kitchenette', '"Lounge area"', 'Desk', '"Reading nook"', '"Smart TV"', '"Mountain view"', '"Stand alone jetted bathtub"', '"Stand alone shower"', '"Stand alone rain shower"', '"Stand alone bathtub"', '"Shower bathtub combo"', '"Rain shower"', '"Jetted tub"', 'Bidet', '"Jet skis"', 'Canoe', '"Stand up paddle board"', '"Boogie boards"', '"Snorkeling

In [151]:
cols = list2[list2['amenities_count'] == 131]['amenities'].values

In [152]:
cols

array([list(['{TV', '"Cable TV"', 'Internet', 'Wifi', '"Air conditioning"', 'Pool', 'Kitchen', '"Free parking on premises"', 'Gym', 'Elevator', '"Garage parking"', '"Hot tub"', '"Indoor fireplace"', '"Suitable for events"', 'Washer', 'Dryer', '"Smoke detector"', '"Carbon monoxide detector"', 'Iron', 'Bathtub', '"Bath towel"', '"Hand or paper towel"', '"Toilet paper"', 'Microwave', '"Coffee maker"', 'Refrigerator', 'Dishwasher', '"Dishes and silverware"', '"Cooking basics"', 'Oven', 'Stove', '"BBQ grill"', '"Patio or balcony"', '"Garden or backyard"', '"Handheld shower head"', '"Ceiling fan"', '"Central air conditioning"', '"Walk in closet"', 'Kitchenette', '"Lounge area"', 'Desk', '"Reading nook"', '"Smart TV"', '"Mountain view"', '"Stand alone jetted bathtub"', '"Stand alone shower"', '"Stand alone rain shower"', '"Stand alone bathtub"', '"Shower bathtub combo"', '"Rain shower"', '"Jetted tub"', 'Bidet', '"Jet skis"', 'Canoe', '"Stand up paddle board"', '"Boogie boards"', '"Snorkeling

In [153]:
amens = list2['amenities'][30400]
amens

['{TV',
 '"Cable TV"',
 'Internet',
 'Wifi',
 '"Air conditioning"',
 'Pool',
 'Kitchen',
 '"Free parking on premises"',
 'Gym',
 'Elevator',
 '"Garage parking"',
 '"Hot tub"',
 '"Indoor fireplace"',
 '"Suitable for events"',
 'Washer',
 'Dryer',
 '"Smoke detector"',
 '"Carbon monoxide detector"',
 'Iron',
 'Bathtub',
 '"Bath towel"',
 '"Hand or paper towel"',
 '"Toilet paper"',
 'Microwave',
 '"Coffee maker"',
 'Refrigerator',
 'Dishwasher',
 '"Dishes and silverware"',
 '"Cooking basics"',
 'Oven',
 'Stove',
 '"BBQ grill"',
 '"Patio or balcony"',
 '"Garden or backyard"',
 '"Handheld shower head"',
 '"Ceiling fan"',
 '"Central air conditioning"',
 '"Walk in closet"',
 'Kitchenette',
 '"Lounge area"',
 'Desk',
 '"Reading nook"',
 '"Smart TV"',
 '"Mountain view"',
 '"Stand alone jetted bathtub"',
 '"Stand alone shower"',
 '"Stand alone rain shower"',
 '"Stand alone bathtub"',
 '"Shower bathtub combo"',
 '"Rain shower"',
 '"Jetted tub"',
 'Bidet',
 '"Jet skis"',
 'Canoe',
 '"Stand up paddl

In [154]:
amenlist = list2['amenities']
amenlist[0]

['{TV',
 '"Cable TV"',
 'Internet',
 'Wifi',
 '"Air conditioning"',
 '"Wheelchair accessible"',
 'Pool',
 'Kitchen',
 '"Free parking on premises"',
 '"Pets allowed"',
 'Gym',
 'Elevator',
 '"Hot tub"',
 '"Indoor fireplace"',
 '"Buzzer/wireless intercom"',
 'Heating',
 '"Family/kid friendly"',
 '"Suitable for events"',
 'Washer',
 'Dryer',
 '"Smoke detector"',
 '"Carbon monoxide detector"',
 '"First aid kit"',
 '"Safety card"',
 '"Fire extinguisher"',
 'Essentials',
 'Shampoo',
 '"24-hour check-in"',
 'Hangers',
 '"Hair dryer"',
 'Iron',
 '"Laptop friendly workspace"}']

In [155]:
pd.DataFrame([amens], columns=amens)


,{TV,"""Cable TV""",Internet,Wifi,"""Air conditioning""",Pool,Kitchen,"""Free parking on premises""",Gym,Elevator,"""Garage parking""","""Hot tub""","""Indoor fireplace""","""Suitable for events""",Washer,Dryer,"""Smoke detector""","""Carbon monoxide detector""",Iron,Bathtub,"""Bath towel""","""Hand or paper towel""","""Toilet paper""",Microwave,"""Coffee maker""",Refrigerator,Dishwasher,"""Dishes and silverware""","""Cooking basics""",Oven,Stove,"""BBQ grill""","""Patio or balcony""","""Garden or backyard""","""Handheld shower head""","""Ceiling fan""","""Central air conditioning""","""Walk in closet""",Kitchenette,"""Lounge area""",Desk,"""Reading nook""","""Smart TV""","""Mountain view""","""Stand alone jetted bathtub""","""Stand alone shower""","""Stand alone rain shower""","""Stand alone bathtub""","""Shower bathtub combo""","""Rain shower""","""Jetted tub""",Bidet,"""Jet skis""",Canoe,"""Stand up paddle board""","""Boogie boards""","""Snorkeling equipment""",Terrace,Balcony,Gazebo,"""Outdoor kitchen""","""Sound system""",Bar,"""Gated property""",Greenhouse,Fountain,"""Alfresco shower""","""Driveway parking""","""Natural gas barbeque""",Sauna,"""Steam room""","""Massage table""","""Wine cellar""","""Security cameras""",Safe,"""Breakfast bar""","""Breakfast table""","""Espresso machine""","""Formal dining area""","""Pool house""","""Sun loungers""","""Pool toys""",Toaster,"""Large mirror""",Garden,"""Private hot tub""","""En suite bathroom""","""Outdoor seating""","""Soaking tub""","""Outdoor parking""","""Walk-in shower""","""Full kitchen""","""Ironing Board""","""Water Filtration System""","""Ice Machine""",Piano,"""Indoor Pool""","""Apple TV""","""Ceiling fans""","""Chef's kitchen""","""Cigar room""","""Decorative fireplace""","""Dining area""","""Foosball table""","""Game room""","""Gas fireplace""","""Home theater""",Kayaks,"""Keurig coffee machine""","""Massage bed""","""Massage room""","""Media room""","""Nespresso machine""",Patio,"""Ping pong table""","""Pool table""","""Powder room""",Projector,"""Safety deposit box""","""Satellite TV""","""Security system""",Shuffleboard,"""Smart home technology""","""Sonos sound system""","""Spa room""","""Surround sound system""","""Wet bar""",Parking,Office,"""Racquetball court""","""Basketball court""}"
0,{TV,"""Cable TV""",Internet,Wifi,"""Air conditioning""",Pool,Kitchen,"""Free parking on premises""",Gym,Elevator,"""Garage parking""","""Hot tub""","""Indoor fireplace""","""Suitable for events""",Washer,Dryer,"""Smoke detector""","""Carbon monoxide detector""",Iron,Bathtub,"""Bath towel""","""Hand or paper towel""","""Toilet paper""",Microwave,"""Coffee maker""",Refrigerator,Dishwasher,"""Dishes and silverware""","""Cooking basics""",Oven,Stove,"""BBQ grill""","""Patio or balcony""","""Garden or backyard""","""Handheld shower head""","""Ceiling fan""","""Central air conditioning""","""Walk in closet""",Kitchenette,"""Lounge area""",Desk,"""Reading nook""","""Smart TV""","""Mountain view""","""Stand alone jetted bathtub""","""Stand alone shower""","""Stand alone rain shower""","""Stand alone bathtub""","""Shower bathtub combo""","""Rain shower""","""Jetted tub""",Bidet,"""Jet skis""",Canoe,"""Stand up paddle board""","""Boogie boards""","""Snorkeling equipment""",Terrace,Balcony,Gazebo,"""Outdoor kitchen""","""Sound system""",Bar,"""Gated property""",Greenhouse,Fountain,"""Alfresco shower""","""Driveway parking""","""Natural gas barbeque""",Sauna,"""Steam room""","""Massage table""","""Wine cellar""","""Security cameras""",Safe,"""Breakfast bar""","""Breakfast table""","""Espresso machine""","""Formal dining area""","""Pool house""","""Sun loungers""","""Pool toys""",Toaster,"""Large mirror""",Garden,"""Private hot tub""","""En suite bathroom""","""Outdoor seating""","""Soaking tub""","""Outdoor parking""","""Walk-in shower""","""Full kitchen""","""Ironing Board""","""Water Filtration System""","""Ice Machine""",Piano,"""Indoor Pool""","""Apple TV""","""Ceiling fans""","""C

In [156]:
amen_df = pd.DataFrame([amens], columns=amens).isin(amenlist[0]).astype(int)
for i in amenlist[1:]:
    tempcolumn = pd.DataFrame([amens], columns=amens).isin(i).astype(int)
    amen_df = pd.concat([amen_df,tempcolumn]) 

In [167]:
amen_df

,{TV,"""Cable TV""",Internet,Wifi,"""Air conditioning""",Pool,Kitchen,"""Free parking on premises""",Gym,Elevator,"""Garage parking""","""Hot tub""","""Indoor fireplace""","""Suitable for events""",Washer,Dryer,"""Smoke detector""","""Carbon monoxide detector""",Iron,Bathtub,"""Bath towel""","""Hand or paper towel""","""Toilet paper""",Microwave,"""Coffee maker""",Refrigerator,Dishwasher,"""Dishes and silverware""","""Cooking basics""",Oven,Stove,"""BBQ grill""","""Patio or balcony""","""Garden or backyard""","""Handheld shower head""","""Ceiling fan""","""Central air conditioning""","""Walk in closet""",Kitchenette,"""Lounge area""",Desk,"""Reading nook""","""Smart TV""","""Mountain view""","""Stand alone jetted bathtub""","""Stand alone shower""","""Stand alone rain shower""","""Stand alone bathtub""","""Shower bathtub combo""","""Rain shower""","""Jetted tub""",Bidet,"""Jet skis""",Canoe,"""Stand up paddle board""","""Boogie boards""","""Snorkeling equipment""",Terrace,Balcony,Gazebo,"""Outdoor kitchen""","""Sound system""",Bar,"""Gated property""",Greenhouse,Fountain,"""Alfresco shower""","""Driveway parking""","""Natural gas barbeque""",Sauna,"""Steam room""","""Massage table""","""Wine cellar""","""Security cameras""",Safe,"""Breakfast bar""","""Breakfast table""","""Espresso machine""","""Formal dining area""","""Pool house""","""Sun loungers""","""Pool toys""",Toaster,"""Large mirror""",Garden,"""Private hot tub""","""En suite bathroom""","""Outdoor seating""","""Soaking tub""","""Outdoor parking""","""Walk-in shower""","""Full kitchen""","""Ironing Board""","""Water Filtration System""","""Ice Machine""",Piano,"""Indoor Pool""","""Apple TV""","""Ceiling fans""","""Chef's kitchen""","""Cigar room""","""Decorative fireplace""","""Dining area""","""Foosball table""","""Game room""","""Gas fireplace""","""Home theater""",Kayaks,"""Keurig coffee machine""","""Massage bed""","""Massage room""","""Media room""","""Nespresso machine""",Patio,"""Ping pong table""","""Pool table""","""Powder room""",Projector,"""Safety deposit box""","""Satellite TV""","""Security system""",Shuffleboard,"""Smart home technology""","""Sonos sound system""","""Spa room""","""Surround sound system""","""Wet bar""",Parking,Office,"""Racquetball court""","""Basketball court""}"
0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [168]:
pd.set_option('display.max_rows', None)

In [169]:
amen_df.mean().sort_values(ascending=False)

"Smoke detector"                0.921844
Kitchen                         0.873969
Wifi                            0.837508
{TV                             0.812685
"Carbon monoxide detector"      0.786891
"Air conditioning"              0.768822
Washer                          0.756162
Dryer                           0.743950
Iron                            0.714393
"Free parking on premises"      0.611738
Refrigerator                    0.509956
Microwave                       0.472125
"Dishes and silverware"         0.465196
"Coffee maker"                  0.441371
"Cooking basics"                0.405483
Oven                            0.397732
Stove                           0.385321
"Cable TV"                      0.284885
Dishwasher                      0.270455
"Patio or balcony"              0.257769
Pool                            0.227290
"Indoor fireplace"              0.217745
Internet                        0.204984
"Garden or backyard"            0.196910
Elevator        